# Subsystem SCQubits Check #

The following file compares the eigenenergies of each static subsystem hamiltonian against the SCQubits implementations of the same circuits

## Filepath and Dependency Setup

In [1]:
import sys
sys.path.append('../src')
import matplotlib.pyplot as plt
import qutip as qt
import scqubits as scq
import numpy as np
import static_system 
from state_labeling import get_dressed_comp_states
import yaml
from subsystems import Subsystem
from composite_systems import CompositeSystem
from typing import Union

## Custom Implementation ##

In [2]:
# extract yaml params
yaml_path = '../config/circuit_parameters.yaml'

with open(yaml_path,'r') as stream:
    try:
        ct_params:dict[str,dict[str,float]] = yaml.safe_load(stream) 
    except yaml.YAMLError as exc:
        print(exc)

#build systems
sys = static_system.build_bare_system(ct_params, stable_nlev=6)

#extract individual circuit elements
flux_A:Subsystem = sys.subsystems['A']
flux_B:Subsystem = sys.subsystems['B']
transmon:Subsystem = sys.subsystems['C']

#extract eigenvalues
flux_A_energies = flux_A.H.eigenenergies()
flux_B_energies = flux_B.H.eigenenergies()
transmon_energies= transmon.H.eigenenergies()

KeyError: 'A'

## SC Qubits Implementation

In [ ]:
# extract parameters
transmon_params = ct_params['C']
flux_A_params = ct_params['A']
flux_B_params = ct_params['B']

# convert to scq parameters
def custom_to_scq_params(my_params:dict[str,float],
                         circuit:str)->dict[str,float]:
    assert circuit in ['fluxonium','transmon'],'wrong circuit param'
    if circuit=='fluxonium':
        scq_params:dict[str,float] = {
            'EC': my_params['E_C']*2*np.pi, 
            'EJ': my_params['E_J']*2*np.pi,
            'EL': my_params['E_L']*2*np.pi,
            'flux': my_params['phi_ext'],
            'cutoff':100
        }
    else:
        w:float = my_params['w']
        U:float = my_params['U']
        scq_params:dict[str,float] = {
            'EJ': (w+2*U)**2/(16*U),
            'EC': 2*U,
            'ng': 0,
            'ncut':3600
        }
    return scq_params

flux_A_scq_params = custom_to_scq_params(flux_A_params, 'fluxonium')
flux_B_scq_params = custom_to_scq_params(flux_B_params, 'fluxonium')
transmon_scq_params = custom_to_scq_params(transmon_params, 'transmon')

# build circuits

flux_A_scq:scq.Fluxonium = scq.Fluxonium(**flux_A_scq_params)
flux_B_scq:scq.Fluxonium = scq.Fluxonium(**flux_B_scq_params)
transmon_scq:scq.Transmon = scq.Transmon(**transmon_scq_params)

# get eigenvalues
flux_A_scq_energies = flux_A_scq.eigenvals()
flux_B_scq_energies = flux_B_scq.eigenvals()
transmon_scq_energies = transmon_scq.eigenvals()
        

## Check that self-implementation and scq implementation agree on transmon eigenenergies

Test passes if the following cell evaluates to true

In [ ]:
tare_transmon_scq_en = transmon_scq_energies - transmon_scq_energies[0]
tare_self_transmon_en = transmon_energies - transmon_energies[0]
#the following should evaluate to true
(np.abs(tare_self_transmon_en - tare_transmon_scq_en)<=1e-3).all()

## Check that self-implementation and scq implementation agree on fluxonium eigenenergies
Test passes if the following two cells evaluate to true

In [ ]:
flux_A_scq_tared_en = flux_A_scq_energies - flux_A_scq_energies[0]
flux_A_tared_en = flux_A_energies - flux_A_energies[0]
(np.abs(flux_A_scq_tared_en - flux_A_tared_en)<=1e-3).all()

In [ ]:
flux_B_scq_tared_en = flux_B_scq_energies - flux_B_scq_energies[0]
flux_B_tared_en = flux_B_energies - flux_B_energies[0]
(np.abs(flux_B_scq_tared_en - flux_B_tared_en)<=1e-3).all()

# Composite System SCQ Check

Define SCQ Composite System

In [ ]:
n_transmon = transmon_scq.n_operator()
n_A = flux_A_scq.n_operator()
n_B = flux_B_scq.n_operator()
interaction_coef = ct_params['interaction']
scq_hilbertspace = scq.HilbertSpace([flux_A_scq, flux_B_scq, transmon_scq])
 #add three interaction terms
scq_hilbertspace.add_interaction(
    g=interaction_coef['g_AB']*2*np.pi,
    op1=(n_A, flux_A_scq),
    op2=(n_transmon, transmon_scq)
)
scq_hilbertspace.add_interaction(
    g=interaction_coef['g_BC']*2*np.pi,
    op1=(n_B, flux_B_scq),
    op2=(n_transmon, transmon_scq)
)
scq_hilbertspace.add_interaction(
    g=interaction_coef['g_AB']*2*np.pi,
    op1=(n_A, flux_A_scq),
    op2=(n_B, flux_B_scq)
)


Extract SCQ and Handrolled Eigenenergies

In [ ]:
scq_dressed_hamiltonian = scq_hilbertspace.hamiltonian()
my_dressed_hamiltonian = sys.H 
scq_eigenenergies = scq_dressed_hamiltonian.eigenenergies()
#scq_eigenenergies -= scq_eigenenergies[0]
my_eigenenergies = my_dressed_hamiltonian.eigenenergies()
#my_eigenenergies -= my_eigenenergies[0]

print(scq_eigenenergies)
print(my_eigenenergies)

# Subsystem SCQ Plots

For Flux A

In [ ]:
fig, ax = flux_A_scq.plot_wavefunction(range(5))
fig.savefig('../plots/flux_A_wavefunction_scq.png')

In [ ]:
fig, ax = transmon_scq.plot_wavefunction(range(5))
fig.savefig('../plots/transmon_wavefunction_scq.png')

In [ ]:
comp_states, comp_coords = get_dressed_comp_states(sys)

ee0_idx = comp_coords[1,1,0]
ge1_idx = comp_coords[0,1,1]
gf0_idx = comp_coords[0,2,0]
eg0_idx = comp_coords[1,0,0]
gg1_idx = comp_coords[0,0,1]
ge0_idx = comp_coords[0,1,0]

w_ee0_ge1 = (my_eigenenergies[ge1_idx] - my_eigenenergies[ee0_idx])/(2*np.pi)
w_gf0_ge1 = (my_eigenenergies[ge1_idx] - my_eigenenergies[gf0_idx])/(2*np.pi)

w_eg0_gg1 = (my_eigenenergies[gg1_idx] - my_eigenenergies[eg0_idx])/(2*np.pi)
w_ge0_gg1 = (my_eigenenergies[gg1_idx] - my_eigenenergies[ge0_idx])/(2*np.pi)

In [ ]:
print(w_ee0_ge1)
print(w_gf0_ge1)
print(w_eg0_gg1)
print(w_ge0_gg1)

In [ ]:
diff = -0.84 - w_ee0_ge1 
print(w_gf0_ge1 + diff)
print(w_ee0_ge1 + diff)
print(diff)

In [ ]:
w_ee0_ge1_scq = (scq_eigenenergies[ge1_idx] - scq_eigenenergies[ee0_idx])/((2*np.pi))
w_gf0_ge1_scq = (scq_eigenenergies[ge1_idx] - scq_eigenenergies[gf0_idx])/((2*np.pi))
print(w_ee0_ge1_scq)
print(w_gf0_ge1_scq)